# Extrayendo los datos haciendo Web Scrapping en Coin desk
 

In [27]:
pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
    --------------------------------------- 0.2/9.4 MB 3.3 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/9.4 MB 7.2 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/9.4 MB 7.9 MB/s eta 0:00:02
   ------- -------------------------------- 1.6/9.4 MB 9.5 MB/s eta 0:00:01
   --------- ------------------------------ 2.2/9.4 MB 9.8 MB/s eta 0:00:01
   ----------- ---------------------------- 2.7/9.4 MB 10.0 MB/s eta 0:00:01
   ------------- -------------------------- 3.2/9.4 MB 10.3 MB/s eta 0:00:01
   ---------------- ----------------------- 3.8/9.4 MB 10.4 MB/s eta 0:00:01
   ------------------ --------------------- 4.3/9.4 MB 10.5 MB/s eta 0:00:01
   -------------------- ------------------- 4.8/9.4 MB 10.6 MB/s eta 0:00:01
   ---------------------- ----------------- 5.4/9.4 MB 10.7 MB/s eta 0:00:01
   ------------------------- -------------- 5.9/9.4 MB 10.8 MB/s eta 0:00:01
   ---------

In [41]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
from newspaper import Article
from tqdm import tqdm

# --- Configuración de fechas ---
start_date = datetime(2022, 4, 6)
end_date = datetime(2025, 4, 6)  # Ajusta este rango como necesites

noticias = []

# --- Recorrer fechas ---
for n in tqdm(range((end_date - start_date).days + 1)):
    fecha_actual = start_date + timedelta(days=n)
    fecha_str = fecha_actual.strftime('%Y-%m-%d')
    rss_url = f"https://www.coindesk.com/arc/outboundfeeds/rss/?outputType=xml&days={fecha_str}"

    try:
        response = requests.get(rss_url)
        soup = BeautifulSoup(response.content, 'xml')
        items = soup.find_all('item')[:3]  # Solo las 3 noticias más relevantes

        for item in items:
            url = item.link.text
            titulo = item.title.text

            try:
                articulo = Article(url)
                articulo.download()
                articulo.parse()

                noticias.append({
                    'fecha': fecha_str,
                    'titulo': articulo.title,
                    'url': url,
                    'contenido': articulo.text
                })
            except Exception as e:
                print(f"❌ Error extrayendo contenido de: {url}\n{e}")

    except Exception as e:
        print(f"⚠️ Error al obtener noticias de {fecha_str}: {e}")

# --- Crear DataFrame ---
df = pd.DataFrame(noticias)



100%|██████████| 1097/1097 [1:18:43<00:00,  4.31s/it]


In [42]:
df.to_csv('C:/Repos/Projecto_TRM/Datos/noticias_bitcoin_top3_diarias.csv', index=False)
print(df.head())

        fecha                                             titulo  \
0  2022-04-06  Bitcoin Price (BTC) Falls to $79K; Ether (ETH)...   
1  2022-04-06  Cathie Wood’s ARK Buys Coinbase (COIN) Shares ...   
2  2022-04-06  Bitcoin (BTC) Price Posts Worst Q1 in a Decade...   
3  2022-04-07  Bitcoin Price (BTC) Falls to $79K; Ether (ETH)...   
4  2022-04-07  Cathie Wood’s ARK Buys Coinbase (COIN) Shares ...   

                                                 url  \
0  https://www.coindesk.com/markets/2025/04/06/bi...   
1  https://www.coindesk.com/markets/2025/04/06/ca...   
2  https://www.coindesk.com/markets/2025/04/06/bi...   
3  https://www.coindesk.com/markets/2025/04/06/bi...   
4  https://www.coindesk.com/markets/2025/04/06/ca...   

                                           contenido  
0  "Decoupling" and "safe haven" began to be used...  
1  Cathie Wood’s ARK Investment Management took a...  
2  Bitcoin just notched its worst first quarter i...  
3  "Decoupling" and "safe haven" b